##### Imports:

In [1]:
from utils import separate_for_training, preprocess, calculate_emission_from_chroma, calculate_mu_from_chroma, calculate_transition_probabilites, format_indiv_chroma, predict, get_unique_predicted, calculate_initial_probabilities
import pickle
from chroma import get_chromagram
import pandas as pd
from tqdm import tqdm
import numpy as np
from hmmlearn import hmm

##### Steps:

1. Training / Testing Data Split
2. Create Chromagram from Training Data
3. Create HMM Initialization Components
    - Initial State Probabilities
    - Transition Probability Matrix
    - Mu Value
    - Emission Matrix
4. Create HMM Object
5. Fit / Train HMM

##### Training / Test Data Split:

In [2]:
# Load data and split into training and test
piece_name_dict = preprocess('dataset.pkl')

training_piece_names, _, test_piece_names = separate_for_training(piece_name_dict, 0.8, 0.)

with open(r"dataset.pkl", 'rb') as data:
    midi_data = pickle.load(data)

100%|██████████| 5762/5762 [00:03<00:00, 1699.56it/s]


##### Create Chromagram from Training Data:

In [3]:
song_chromagrams = []
for song_name in tqdm(list(training_piece_names)):
    indiv_chroma = get_chromagram(song_name, midi_data)
    formatted = format_indiv_chroma(indiv_chroma)
    song_chromagrams.append(indiv_chroma)

chromagram = pd.concat(song_chromagrams)
chromagram.head(200)

100%|██████████| 4609/4609 [00:34<00:00, 135.26it/s]


,C,C#,D,D#,E,F,F#,G,G#,A,A#,B,Chord Actual
0,0,0,78,0,0,0,0,111,0,71,0,0,G
1,0,0,78,0,0,0,0,111,0,71,0,0,G
2,0,0,78,0,0,0,0,111,0,71,0,0,G
3,0,0,78,0,0,0,0,111,0,71,0,0,G
4,0,0,78,0,0,0,0,111,0,71,0,0,G
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,0,0,0,44,0,0,48,0,0,0,0,150,B
2,0,0,0,147,0,0,48,0,0,0,0,60,B
3,0,0,0,147,0,0,48,0,0,0,0,60,B
4,0,0,0,147,0,0,48,0,0,0,0,60,B


##### Create HMM Components:

###### Initial State Probabilities:

In [4]:
initial_state_probabilties = calculate_initial_probabilities(training_piece_names, midi_data)
initial_state_probabilties

Niko_Kotoulas_ChordProg_3_G-Em-A-D (IV-ii-V-I).mid
Niko_Kotoulas_Melody_8_Bm-A-G-F#m (vi-V-IV-iii) - 130-160bpm.mid
Niko_Kotoulas_Drop_Pianos_2_E-G#m-F# (IV-vi-V).mid
Niko_Kotoulas__RhythmChordProg_4_B-E-F#-B (I-IV-V-I).mid
Niko_Kotoulas_ChordProg_8_C#m-G#m-B-F# (ii-vi-I-V).mid
Niko_Kotoulas_Melody_9_A-Bm-D-G (V-vi-I-IV) - 160-180bpm.mid
Niko_Kotoulas_ChordProg_8_E-F#-G#m-B-F#-E (IV-V-vi-I-V-IV).mid
Niko_Kotoulas_Melody_8_B-F#-G#m-E (I-V-vi-IV) - 115-130bpm.mid
Niko_Kotoulas_ChordProg_9_Bm-Em-Bm-F#m (vi-ii-vi-iii).mid
Niko_Kotoulas_Cluster_Bm-G-D-A (vi-IV-I-V).mid
Niko_Kotoulas_Melody_6_E-G#m-F#-D#m (IV-vi-V-iii) - 115-130bpm.mid
Niko_Kotoulas_EDM_7th_9th_Chords_12_G#m-F#-E-F#_.mid
Niko_Kotoulas_Melody_13_C#m-G#m-B-F# (ii-vi-I-V) - 160-180bpm.mid
Niko_Kotoulas_ChordProg_1_D#m-E-B-C#m (iii-IV-I-ii).mid
Niko_Kotoulas_Power Octave_G-Bm-A (IV-vi-V).mid
Niko_Kotoulas_Seventh_E-F#-G#m-B-F#-E (IV-V-vi-I-V-IV).mid
Niko_Kotoulas_Melody_5_C#m-G#m-B-F# (ii-vi-I-V) - 160-180bpm.mid
Niko_Kotoulas_M

C        0.001085
Cm       0.000000
Cdim     0.000000
C#       0.000000
C#m      0.027772
C#dim    0.009547
D        0.046648
Dm       0.000000
Ddim     0.000000
D#       0.000000
D#m      0.068562
D#dim    0.000000
E        0.023866
Em       0.140811
Edim     0.000000
F        0.000000
Fm       0.000000
Fdim     0.000000
F#       0.375570
F#m      0.169234
F#dim    0.000000
G        0.000000
Gm       0.000000
Gdim     0.000000
G#       0.000000
G#m      0.000000
G#dim    0.000000
A        0.054459
Am       0.000000
Adim     0.000000
A#       0.000000
A#m      0.000000
A#dim    0.000000
B        0.005858
Bm       0.076589
Bdim     0.000000
dtype: float64

###### Transition Matrix:

In [5]:
transition_prob_matrix = calculate_transition_probabilites(chromagram)
transition_prob_matrix

C:\Users\julia\CS4100\final\CS4100_Final\utils.py:139: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[8.60784314e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.45730108e-04 0.00000000e+00 3.58134121e-04 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 1.54416306e-04 0.00000000e+00 1.17381490e-01
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 4.10088169e-04 0.00000000e+00 0.00000000e+00
 0.00000000e+00 6.51338501e-05 0.00000000e+00 1.44310556e-04
 0.00000000e+00 0.00000000e+00 2.27272727e-02 0.00000000e+00
 0.00000000e+00 0.00000000e+00 1.31449228e-04 0.00000000e+00]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  all_chords_matrix.update(transition_prob_matrix)
C:\Users\julia\CS4100\final\CS4100_Final\utils.py:139: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise i

,C,Cm,Cdim,C#,C#m,C#dim,D,Dm,Ddim,D#,...,G#dim,A,Am,Adim,A#,A#m,A#dim,B,Bm,Bdim
C,0.860784,0.000000,0.000000,0.000000,0.000000,0.001961,0.001961,0.000000,0.000000,0.000000,...,0.000000,0.007843,0.000000,0.000000,0.000000,0.000000,0.000000,0.007843,0.003922,0.000000
Cm,0.027778,0.027778,0.027778,0.027778,0.027778,0.027778,0.027778,0.027778,0.027778,0.027778,...,0.027778,0.027778,0.027778,0.027778,0.027778,0.027778,0.027778,0.027778,0.027778,0.027778
Cdim,0.027778,0.027778,0.027778,0.027778,0.027778,0.027778,0.027778,0.027778,0.027778,0.027778,...,0.027778,0.027778,0.027778,0.027778,0.027778,0.027778,0.027778,0.027778,0.027778,0.027778
C#,0.027778,0.027778,0.027778,0.027778,0.027778,0.027778,0.027778,0.027778,0.027778,0.027778,...,0.027778,0.027778,0.027778,0.027778,0.027778,0.027778,0.027778,0.027778,0.027778,0.027778
C#m,0.000146,0.000000,0.000000,0.000000,0.859204,0.000000,0.001895,0.000000,0.000000,0.000000,...,0.000000,0.000437,0.000000,0.000000,0.000000,0.000000,0.000000,0.009182,0.003352,0.000000
C#dim,0.000000,0.000000,0.000000,0.000000,0.007229,0.881928,0.024096,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.007229,0.016867,0.000000
D,0.000358,0.000000,0.000000,0.000000,0.002149,0.000000,0.863820,0.000000,0.000000,0.000000,...,0.000000,0.053899,0.000000,0.000000,0.000090,0.000000,0.000000,0.003402,0.026860,0.000000
Dm,0.027778,0.027778,0.027778,0.027778,0.027778,0.027778,0.027778,0.027778,0.027778,0.027778,...,0.027778,0.027778,0.027778,0.027778,0.027778,0.027778,0.027778,0.027778,0.027778,0.027778
Ddim,0.027778,0.027778,0.027778,0.027778,0.027778,0.027778,0.027778,0.027778,0.027778,0.027778,...,0.027778,0.027778,0.027778,0.027778,0.027778,0.027778,0.027778,0.027778,0.027778,0.027778
D#,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500,0.000000,0.000000,0.875000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


###### Mu Value:

In [31]:
mu = calculate_mu_from_chroma(chromagram)
mu = mu.to_numpy()

###### Emission Matrix:

In [32]:
emission = calculate_emission_from_chroma(chromagram)

In [42]:
emission
equal = np.array_equal(emission, emission.T)
print(equal)
print(emission.shape)
print(np.linalg.cholesky(emission))

False
(20, 12, 12)


LinAlgError: Matrix is not positive definite

In [35]:
model = hmm.GaussianHMM(n_components=transition_prob_matrix.shape[0], covariance_type="full")
model.startprob_ = initial_state_probabilties
model.transmat_ = transition_prob_matrix.values
model.means_ = mu.reshape(-1, 1)
model.covars_ = emission
print(model.means_.shape)

ValueError: component 1 of 'full' covars must be symmetric, positive-definite

In [36]:
chroma_with_preds = predict(transition_prob_matrix, model, mu)

AttributeError: 'GaussianHMM' object has no attribute '_covars_'